In [223]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random,jieba

In [224]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents_ch.json') as json_data:
    intents = json.load(json_data)

In [225]:
tf.reset_default_graph()
tflearn.config.init_training_mode()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [226]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    #sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    #sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    w = jieba.cut_for_search(sentence)
    w = ("/".join(w)).split("/")
    return w

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [227]:
p = bow("你们每天几点开门", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
['今天租', '付款', '你们哪天方便', '再见', '出租', '助力车', '时间', '明天租', '营业时间', '谢谢', '问候']


In [228]:
# load our saved model

model.load('./model.tflearn',weights_only=True)

INFO:tensorflow:Restoring parameters from /home/mqm/Workspace/dl/notebooks/model.tflearn


In [255]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    #print(results)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        print('random response')
                        return print(random.choice(i['responses']))

            results.pop(0)

In [256]:
classify('几点开门啊')

[('时间', 0.94440132)]

In [257]:
response('几点开门')
#response('Your business hours')

random response
我们营业时间是每天早9点到晚上9点


In [232]:
response('有什么样的助力车')

我们出租雅马哈，比亚乔和维斯帕摩托车


In [233]:
response('刷信用卡吗')

可以


In [234]:
response('我想租个助力车')

你打算今天租，还是周末租


In [235]:
# show context
context

{'123': '出租'}

In [236]:
response('你们哪天方便')

只要提前1天预约，什么时候都可以


In [237]:
context

{'123': '出租'}

In [238]:
response("今天")

如果今天租 请拨打1800


In [239]:
context

{'123': '出租'}

In [240]:
classify('今天')

[('今天租', 0.92587358)]

In [241]:
# clear context
response("太谢谢你了", show_details=True)

tag: 谢谢
太客气了


In [242]:
response('today')
classify('today')

待会见


[('再见', 0.42227998)]

In [243]:
response("thanks, your great")

拜拜，欢迎下次再来


In [128]:
context

{'123': ''}

In [129]:
response('see you tomorrow')

See you later, thanks for visiting
